<h1><u>  Imports</u> </h1>

In [1]:
import numpy as np
import pandas as pd
import ast
from text_metrics.utils import get_metrics, init_tok_n_model
import warnings
warnings.filterwarnings("ignore")
import en_core_web_sm 
nlp = en_core_web_sm.load()

<h1><u>  Load Data</u> </h1>

In [2]:
# load and display the data 
df = pd.read_csv(r'data/ia_A.tsv',sep='\t',encoding='utf-16')
df.head(1000)

,RECORDING_SESSION_LABEL,TRIAL_INDEX,DATA_FILE,EYE_REPORTED,EYE_TRACKED,GROUPING_VARIABLES,IA_AREA,IA_AVERAGE_FIX_PUPIL_SIZE,IA_BOTTOM,IA_DWELL_TIME,...,level_ind,list,paragraph,paragraph_id,practice,principle_list,q_ind,question,reread,trial
0,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,21090.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
1,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,6327.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
2,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,12654.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
3,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,27417.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
4,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,16872.0,.,264,0,...,0,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,l42_2070,28,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,10272.0,.,1173,0,...,2,42,Going out with friends was a problem – no Amer...,4,0,6,0,Why was listening to Daft Punk forbidden?,0,27
996,l42_2070,29,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,10260.0,.,261,0,...,2,42,The documentary shows Carle – breaking his iPh...,5,0,6,2,What does Carle do with his new Tefal saucepan?,0,28
997,l42_2070,29,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,10260.0,.,261,0,...,2,42,The documentary shows Carle – breaking his iPh...,5,0,6,2,What does Carle do with his new Tefal saucepan?,0,28
998,l42_2070,29,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,12312.0,.,261,0,...,2,42,The documentary shows Carle – breaking his iPh...,5,0,6,2,What does Carle do with his new Tefal saucepan?,0,28



<h1><u> Segment the Rows</u> </h1>

In [3]:
def concat_strings(row):
    order = [0,1,2,3]
    concatenated = row['question'] + ' ' + ' '.join([row[['a', 'b', 'c', 'd'][i]] for i in order])
    return concatenated

df['concatenated'] = df.apply(concat_strings, axis=1)

In [4]:


# Apply function to each row to create the dictionary column
def find_sentence_indices(main_list, sub_str):
    sub_list = sub_str.split()
    main_str = ' '.join(main_list)
    sub_str = ' '.join(sub_list)
    
    start_index = main_str.find(sub_str)
    if start_index != -1:
        start_word_index = len(main_str[:start_index].split()) + 1
        end_word_index = start_word_index + len(sub_list) - 1
    else:
        start_word_index = end_word_index = -1
    return (start_word_index, end_word_index)

# Function to concatenate strings based on answer order


# Function to create dictionary for each row with sentence indices
def create_dict(row):
    concatenated_sentence = row['concatenated']
    word_list = concatenated_sentence.split()

    result = {}
    for col in columns_to_check:
        result[col] = find_sentence_indices(word_list, row[col])
    return result

columns_to_check = ['a', 'b', 'c', 'd', 'question']

df['indices_dict'] = df.apply(create_dict, axis=1)


In [5]:
def find_segment(row):
    for key, value in row['indices_dict'].items():
        if value[0] <= row['IA_ID'] <= value[1]:
            return key
    return None

df['segmented'] = df.apply(find_segment, axis=1)
df = df.drop(columns=['concatenated', 'indices_dict'])


In [6]:
df.head(200)

,RECORDING_SESSION_LABEL,TRIAL_INDEX,DATA_FILE,EYE_REPORTED,EYE_TRACKED,GROUPING_VARIABLES,IA_AREA,IA_AVERAGE_FIX_PUPIL_SIZE,IA_BOTTOM,IA_DWELL_TIME,...,list,paragraph,paragraph_id,practice,principle_list,q_ind,question,reread,trial,segmented
0,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,21090.0,.,264,0,...,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1,question
1,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,6327.0,.,264,0,...,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1,question
2,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,12654.0,.,264,0,...,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1,question
3,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,27417.0,.,264,0,...,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1,question
4,l42_2070,1,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,16872.0,.,264,0,...,42,Leading water scientists have issued one of th...,1,1,6,0,"According to Malik Falkenmark's report, what w...",0,1,question
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,l42_2070,6,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,14231.0,1120.00,831,166,...,42,Electric bikes are different from motorcycles ...,3,0,6,0,What is one difference between e-bikes and mop...,0,5,b
196,l42_2070,6,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,8132.0,.,831,0,...,42,Electric bikes are different from motorcycles ...,3,0,6,0,What is one difference between e-bikes and mop...,0,5,b
197,l42_2070,6,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,16371.0,1161.00,831,184,...,42,Electric bikes are different from motorcycles ...,3,0,6,0,What is one difference between e-bikes and mop...,0,5,b
198,l42_2070,6,l42_2070.edf,RIGHT,Right,RECORDING_SESSION,16720.0,1053.33,834,504,...,42,Electric bikes are different from motorcycles ...,3,0,6,0,What is one difference between e-bikes and mop...,0,5,c


<h1> <u> Reomve Redundant Columns </u> </h1>

Our main objective with the data is to predict the answer a reader would choose using eye movements.<br>
We were also tasked with deciding each word to what part it belongs (which answer/question). <br> 
Thus, we decided to keep only the relevant columns.

In [7]:
# select which columns to remove from the df
uninformative_columns = [col for col in df.columns if df[col].nunique() == 1]

# batch condition - p=preview =huntingנ
columns_to_remove = ['DATA_FILE','EYE_REPORTED','GROUPING_VARIABLES','IA_AVERAGE_FIX_PUPIL_SIZE','IA_DYNAMIC','IA_END_TIME','IA_GROUP','IA_INSTANCES_COUNT','IA_MAX_FIX_PUPIL_SIZE','IA_MIN_FIX_PUPIL_SIZE','IA_POINTS','IA_START_TIME','IA_TYPE','IP_END_EVENT_MATCHED','IP_END_TIME','IP_INDEX','IP_LABEL','IP_START_EVENT_MATCHED','IP_START_TIME','REPORTING_METHOD','TRIAL_LABEL','COMPREHENSION_PERCENT','COMPREHENSION_SCORE','CONFIRM_FINAL_ANSWER_RT','DUMMY','NUM_RECALIBRATIONS','RECALIBRATE','Session_Name_','Trial_Index_','Trial_Recycled_','article_title','level','paragraph','dspan_inds','aspan_inds','a', 'b', 'c', 'd', 'question']
df_reduced = df.drop(columns=uninformative_columns)
df_reduced = df.drop(columns=columns_to_remove)

df_reduced.head(200)

,RECORDING_SESSION_LABEL,TRIAL_INDEX,EYE_TRACKED,IA_AREA,IA_BOTTOM,IA_DWELL_TIME,IA_DWELL_TIME_%,IA_FIRST_FIXATION_DURATION,IA_FIRST_FIXATION_INDEX,IA_FIRST_FIXATION_PREVIOUS_FIX_IA,...,has_preview,level_ind,list,paragraph_id,practice,principle_list,q_ind,reread,trial,segmented
0,l42_2070,1,Right,21090.0,264,0,0.0000,.,.,.,...,1,0,42,1,1,6,0,0,1,question
1,l42_2070,1,Right,6327.0,264,0,0.0000,.,.,.,...,1,0,42,1,1,6,0,0,1,question
2,l42_2070,1,Right,12654.0,264,0,0.0000,.,.,.,...,1,0,42,1,1,6,0,0,1,question
3,l42_2070,1,Right,27417.0,264,0,0.0000,.,.,.,...,1,0,42,1,1,6,0,0,1,question
4,l42_2070,1,Right,16872.0,264,0,0.0000,.,.,.,...,1,0,42,1,1,6,0,0,1,question
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,l42_2070,6,Right,14231.0,831,166,0.0364,166,24,.,...,1,2,42,3,0,6,0,0,5,b
196,l42_2070,6,Right,8132.0,831,0,0.0000,.,.,.,...,1,2,42,3,0,6,0,0,5,b
197,l42_2070,6,Right,16371.0,831,184,0.0403,184,25,24,...,1,2,42,3,0,6,0,0,5,b
198,l42_2070,6,Right,16720.0,834,504,0.1105,151,9,18,...,1,2,42,3,0,6,0,0,5,c


<h1> <u> Remove Redundent Rows </u> </h1>


In the meetings we said we need to focus on the passages without reread and that weren't practice

In [8]:
df_reduced = df_reduced[df_reduced['reread'] == 0]
df_reduced = df_reduced[df_reduced['practice'] == 0]
df_reduced.drop(columns=['reread', 'practice'], inplace=True)
df_reduced.head(200)


,RECORDING_SESSION_LABEL,TRIAL_INDEX,EYE_TRACKED,IA_AREA,IA_BOTTOM,IA_DWELL_TIME,IA_DWELL_TIME_%,IA_FIRST_FIXATION_DURATION,IA_FIRST_FIXATION_INDEX,IA_FIRST_FIXATION_PREVIOUS_FIX_IA,...,correct_answer,d_key,has_preview,level_ind,list,paragraph_id,principle_list,q_ind,trial,segmented
99,l42_2070,4,Right,10165.0,261,0,0.0000,.,.,.,...,3,2,1,2,42,1,6,2,3,question
100,l42_2070,4,Right,6099.0,261,163,0.0277,163,1,.,...,3,2,1,2,42,1,6,2,3,question
101,l42_2070,4,Right,14231.0,261,0,0.0000,.,.,.,...,3,2,1,2,42,1,6,2,3,question
102,l42_2070,4,Right,28569.0,261,0,0.0000,.,.,.,...,3,2,1,2,42,1,6,2,3,question
103,l42_2070,4,Right,4180.0,492,184,0.0313,184,2,2,...,3,2,1,2,42,1,6,2,3,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,l42_2070,8,Right,8547.0,1287,0,0.0000,.,.,.,...,2,0,1,2,42,5,6,1,7,d
295,l42_2070,9,Right,8208.0,261,0,0.0000,.,.,.,...,0,1,1,0,42,6,6,0,8,question
296,l42_2070,9,Right,8208.0,261,0,0.0000,.,.,.,...,0,1,1,0,42,6,6,0,8,question
297,l42_2070,9,Right,8208.0,261,0,0.0000,.,.,.,...,0,1,1,0,42,6,6,0,8,question


<h1> <u> Preprocessing for Analysis </u> </h1>
Like we talked in the meeting there are servel things we would like to do to the data now.

We wanted to segment what part (i.e. a,b,c,d, or question)  each fixation belonged to,so we could infer the order in which a reader read the answers,in order to produce the graphs in the Analysis.ipynb file.

In [9]:
# Create a new column 'section' that identifies sections based on 'IA_ID'
df_reduced['section'] = (df_reduced['IA_ID'] == 1).cumsum()

# Initialize the 'reading_order_Segmentation' column with NaN values
df_reduced['reading_order_Segmentation'] = np.nan
df_reduced['ordered_reading_times'] = np.nan

df_reduced['reading_order'] = np.nan

# Iterate over each group in 'section' to process data within each section
for section, section_df in df_reduced.groupby('section'): 
    seen = set()
    new_list = []
    order = []
    times = []

    # Create a lookup DataFrame for the current section
    lookup_df = section_df[['IA_ID', 'segmented','INTEREST_AREA_FIXATION_SEQUENCE','IA_DWELL_TIME']].set_index('IA_ID')
    #  Extract and parse the fixation sequence for the first row
    INTEREST_AREA_FIXATION_SEQUENCE = ast.literal_eval(lookup_df['INTEREST_AREA_FIXATION_SEQUENCE'][1])
    # Construct a new list based on the fixation sequence
    for item in INTEREST_AREA_FIXATION_SEQUENCE:
        if item == 0:
            new_list.append(0)
            item_letter = 0
        else:
            new_list.append(lookup_df.loc[item, 'segmented'])
            item_letter = lookup_df.loc[item, 'segmented']
            times.append(lookup_df.loc[item, 'IA_DWELL_TIME'])

        if  item_letter not in seen and item_letter in ['a', 'b', 'c', 'd']: 
            order.append(item_letter)
            seen.add(item_letter)
    
    # Create a boolean mask to identify rows in the current section
    mask = df_reduced['section'] == section
    
    # Update 'reading_order_Segmentation' for the rows in the current section
    df_reduced.loc[mask, 'reading_order_Segmentation'] = pd.Series([new_list] * mask.sum(), index=df_reduced.index[mask])
    df_reduced.loc[mask, 'reading_order'] = pd.Series([order] * mask.sum(), index=df_reduced.index[mask])
    df_reduced.loc[mask, 'ordered_reading_times'] = pd.Series([times] * mask.sum(), index=df_reduced.index[mask])



In [10]:
df_reduced.head(200)

,RECORDING_SESSION_LABEL,TRIAL_INDEX,EYE_TRACKED,IA_AREA,IA_BOTTOM,IA_DWELL_TIME,IA_DWELL_TIME_%,IA_FIRST_FIXATION_DURATION,IA_FIRST_FIXATION_INDEX,IA_FIRST_FIXATION_PREVIOUS_FIX_IA,...,list,paragraph_id,principle_list,q_ind,trial,segmented,section,reading_order_Segmentation,ordered_reading_times,reading_order
99,l42_2070,4,Right,10165.0,261,0,0.0000,.,.,.,...,42,1,6,2,3,question,1,"[question, a, a, a, a, c, c, c, d, d, b, b, b,...","[163, 184, 363, 435, 229, 225, 716, 350, 316, ...","[a, c, d, b]"
100,l42_2070,4,Right,6099.0,261,163,0.0277,163,1,.,...,42,1,6,2,3,question,1,"[question, a, a, a, a, c, c, c, d, d, b, b, b,...","[163, 184, 363, 435, 229, 225, 716, 350, 316, ...","[a, c, d, b]"
101,l42_2070,4,Right,14231.0,261,0,0.0000,.,.,.,...,42,1,6,2,3,question,1,"[question, a, a, a, a, c, c, c, d, d, b, b, b,...","[163, 184, 363, 435, 229, 225, 716, 350, 316, ...","[a, c, d, b]"
102,l42_2070,4,Right,28569.0,261,0,0.0000,.,.,.,...,42,1,6,2,3,question,1,"[question, a, a, a, a, c, c, c, d, d, b, b, b,...","[163, 184, 363, 435, 229, 225, 716, 350, 316, ...","[a, c, d, b]"
103,l42_2070,4,Right,4180.0,492,184,0.0313,184,2,2,...,42,1,6,2,3,a,1,"[question, a, a, a, a, c, c, c, d, d, b, b, b,...","[163, 184, 363, 435, 229, 225, 716, 350, 316, ...","[a, c, d, b]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,l42_2070,8,Right,8547.0,1287,0,0.0000,.,.,.,...,42,5,6,1,7,d,5,"[question, a, a, a, a, c, c, 0, d, c, c, c, b,...","[215, 151, 166, 161, 193, 338, 234, 120, 338, ...","[a, c, d, b]"
295,l42_2070,9,Right,8208.0,261,0,0.0000,.,.,.,...,42,6,6,0,8,question,6,"[question, a, b, d, 0, c, d, 0, b, b, a]","[152, 300, 370, 352, 152, 352, 370, 75, 300]","[a, b, d, c]"
296,l42_2070,9,Right,8208.0,261,0,0.0000,.,.,.,...,42,6,6,0,8,question,6,"[question, a, b, d, 0, c, d, 0, b, b, a]","[152, 300, 370, 352, 152, 352, 370, 75, 300]","[a, b, d, c]"
297,l42_2070,9,Right,8208.0,261,0,0.0000,.,.,.,...,42,6,6,0,8,question,6,"[question, a, b, d, 0, c, d, 0, b, b, a]","[152, 300, 370, 352, 152, 352, 370, 75, 300]","[a, b, d, c]"


We want to normalize the reading times by reading speed and answer/question length so they will be comparable.

In [11]:
# Function 1: Normalize by Counting Rows with Same Section and Segmented Values
def normalize_by_segmented_words(df):
    def normalize_section(section_df):
        # Count the number of rows with the same section and segmented value
        word_count = section_df.groupby(['section', 'segmented'])['IA_ID'].transform('count')
        # Sum IA_DWELL_TIME for the same section and segmented value
        total_dwell_time = section_df.groupby(['section', 'segmented'])['IA_DWELL_TIME'].transform('sum')
        # Normalize by dividing total dwell time by the word count
        section_df['average_reading_time_by_length'] = total_dwell_time / word_count
        # Add a new column with the word count
        section_df['word_count_by_segment'] = word_count
        
        return section_df

    # Apply the normalization to the whole dataframe, grouped by section
    return df.groupby('section').apply(normalize_section).reset_index(drop=True)

df_reduced = normalize_by_segmented_words(df_reduced)


In [12]:
# Function 2: Normalize by Reading Speed for Questions
def normalize_by_reading_speed(df):
    def normalize_section(section_df):
        if section_df['batch_condition'].iloc[0] != 'p':
            # Calculate reading speed for 'question' rows within the same section
            question_df = section_df[section_df['segmented'] == 'question']
            if not question_df.empty:
                # Reading speed = total time / number of words (number of rows in this case)
                reading_speed = question_df['IA_DWELL_TIME'].sum() / question_df['IA_ID'].count()

                # Sum IA_DWELL_TIME for each segmented value in the same section
                section_df['segmented_sum_time'] = section_df.groupby(['section', 'segmented'])['IA_DWELL_TIME'].transform('sum')

                # Normalize the summed time by the reading speed
                section_df['average_reading_time_by_speed'] = section_df['segmented_sum_time'] / reading_speed

                # Only apply the normalization to rows where segmented != 'question'
                section_df.loc[section_df['segmented'] == 'question', 'normalized_time'] = None  # or any other placeholder to indicate no normalization
        return section_df

    # Apply the normalization to the whole dataframe, grouped by section
    return df.groupby('section').apply(normalize_section).reset_index(drop=True)
df_reduced = normalize_by_reading_speed(df_reduced)


In [13]:
df_reduced = df_reduced.drop(columns=['segmented_sum_time', 'normalized_time'])


We want to add surprisal and frequency to each word for further analysis.

In [14]:
unique_texts = set()
for col in ['a', 'b', 'c', 'd', 'question']:
    unique_texts.update(df[col].unique())

# Create a single text string by joining the unique values
text = " ".join(unique_texts)


In [15]:
# Initialize models and tokenizers
model_names = ["gpt2-medium"]

models_tokenizers = [init_tok_n_model(model_name) for model_name in model_names]
tokenizers = [tokenizer for tokenizer, _ in models_tokenizers]
models = [model for _, model in models_tokenizers]

In [16]:
# Get metrics using the constructed text
surp_res = get_metrics(
    text=text,
    models=models,
    tokenizers=tokenizers,
    model_names=model_names,      
    parsing_model=en_core_web_sm.load(),
    add_parsing_features=False,
)


Token indices sequence length is longer than the specified maximum sequence length for this model (21951 > 1024). Running this sequence through the model will result in indexing errors


The context length is too long (21951>1024) for gpt2-medium. Splitting the sentence into chunks with stride 512


In [17]:
df_reduced_temp = df_reduced[["IA_LABEL","segmented","section","IA_ID"]]

In [18]:
df_temp =pd.DataFrame(surp_res)


In [ ]:
merged_df = pd.merge(df_reduced_temp,df_temp, left_on='IA_LABEL', right_on='Word', how='inner')
merged_df.drop(['Word','Length'], axis=1, inplace=True)
merged_df.head(200)

In [ ]:
merged_df = merged_df.sort_values(by=["section", "IA_ID"])


<h1> <u> Save Preprocessed Data </u> </h1>


In [21]:
# Save DataFrame to a TSV file
df_reduced.to_csv('ia_A_preprocessed.tsv', sep='\t', index=False)
merged_df.to_csv('word_statistics.tsv', sep='\t', index=False)
